In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json

from keras.models import model_from_json
from urllib.request import urlopen
from PIL import Image

Using TensorFlow backend.


In [2]:
with open('model_animal_10.json','r') as f:
    model_json = json.load(f)

model = model_from_json(json.dumps(model_json))
model.load_weights('model_animal_10.h5')


In [3]:
def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized

In [ ]:
img = input('Input image link : ')
img_ = [img]
test_images = []
for animal in img_:
    img = Image.open(urlopen(animal)).convert('RGB') 
    img = np.array(img) 
    # Convert RGB to BGR 
    img = img[:, :, ::-1].copy() 

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))

    #out put 224*224px 
    img = img[16:240, 16:240]
    test_images.append(img)
    plt.imshow(img)
    plt.show()

test_images = np.array(test_images).reshape(-1, 224, 224, 3)
something = model.predict(test_images)
animals = ['sapi', 'gajah', 'kucing', 'laba-laba', 'ayam', 'kupu-kupu', 'domba', 'kuda', 'musang', 'anjing']
for pred in something:
    print('Predict :', animals[np.where(pred.max() == pred)[0][0]])
